In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoConfig
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding
from tokenizers import AddedToken
from datasets import Dataset, features
import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GroupKFold

In [8]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
master = pd.read_csv("data/clothing_master.csv")

train = pd.merge(train,master, on = "Clothing ID",how = "left")
test = pd.merge(test,master, on = "Clothing ID",how = "left")
sample_submission = pd.read_csv("data/atmaCup17__sample_submission.csv")

In [9]:
# ====================================================
# Configurations
# ====================================================
class CFG:
    VER = 1
    MODEL_PATH = "microsoft/deberta-v3-large"
    MAX_LENGTH = 256
    STEPS = 25
    USE_GPU = torch.cuda.is_available()
    SEED = 0
    N_SPLIT = 3
    target_col = 'Recommended IND'
    target_col_class_num = 2
    metric = 'auc'
    metric_maximize_flag = True

- Clothing ID: 衣服のID
- Age: レビューをしている人の年齢
- Title: レビューのタイトル
- Review Text: レビューの本文
- Rating: レビューでつけた点数。この列は学習用データでのみ提供されます。
- Recommended IND: レビューした人が、この商品をおすすめするかどうか。今回のコンペではこの値が1になる確率を予測してください。学習用データでのみ提供されます。
- Positive Feedback Count: 他のユーザーからつけられたイイネの数。

In [10]:
category = ["Division Name","Department Name","Class Name"]

# OneHotEncodeしたい列を指定。Nullや不明の場合の補完方法も指定。
ce_ohe = ce.OneHotEncoder(cols=category,handle_unknown='impute', use_cat_names=True)
ce_ohe.fit(train[category])


train_dummy = ce_ohe.transform(train[category])
test_dummy = ce_ohe.transform(test[category])

train = pd.concat([train, train_dummy],axis=1)
test  = pd.concat([test, test_dummy],axis=1)

In [11]:
def preprocessing(df):
    df['Title'] = df['Title'].fillna('')
    df['Review Text'] = df['Review Text'].fillna('')
    df['prompt'] = df['Title'] + ' ' + df['Review Text']
    return df
train_df = preprocessing(train)
test_df = preprocessing(test)
train_df['labels'] = train_df["Recommended IND"].astype(np.int8)

In [12]:
train_df.columns

Index(['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name', 'Division Name_General',
       'Division Name_Initmates', 'Division Name_General Petite',
       'Department Name_Bottoms', 'Department Name_Tops',
       'Department Name_Dresses', 'Department Name_Intimate',
       'Department Name_Jackets', 'Department Name_Trend', 'Class Name_Skirts',
       'Class Name_Pants', 'Class Name_Sweaters', 'Class Name_Dresses',
       'Class Name_Blouses', 'Class Name_Swim', 'Class Name_Knits',
       'Class Name_Fine gauge', 'Class Name_Outerwear', 'Class Name_Jeans',
       'Class Name_Shorts', 'Class Name_Lounge', 'Class Name_Jackets',
       'Class Name_Layering', 'Class Name_Trend', 'Class Name_Legwear',
       'prompt', 'labels'],
      dtype='object')

In [13]:
feature = ['Clothing ID', 'Age', 'Positive Feedback Count',
             'Division Name_General', 'Division Name_Initmates', 'Division Name_General Petite',
             'Department Name_Bottoms', 'Department Name_Tops', 'Department Name_Dresses', 'Department Name_Intimate', 'Department Name_Jackets', 'Department Name_Trend',
             'Class Name_Skirts', 'Class Name_Pants', 'Class Name_Sweaters', 'Class Name_Dresses', 'Class Name_Blouses', 'Class Name_Swim', 'Class Name_Knits', 'Class Name_Fine gauge', 'Class Name_Outerwear', 'Class Name_Jeans', 'Class Name_Shorts', 'Class Name_Lounge', 'Class Name_Jackets', 'Class Name_Layering', 'Class Name_Trend', 'Class Name_Legwear']

train_features = train_df[feature].values.tolist()
test_features = test_df[feature].values.tolist()

In [14]:
from sklearn import preprocessing
mm = preprocessing.MinMaxScaler()
mm.fit(train_features)

train_features = mm.transform(train_features)
test_features = mm.transform(test_features)

# models

In [16]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
model_name = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

def tokenize(sample):
    return tokenizer(sample['prompt'], max_length=CFG.MAX_LENGTH, truncation=True)

# metricをAUCに変更
def compute_metrics(p):
    preds, labels = p
    preds = torch.softmax(torch.tensor(preds), dim = 1).numpy()
    score = roc_auc_score(labels, preds[:, 1])
    return {'auc': score}

ds_test = Dataset.from_pandas(test_df[['prompt']].copy())
ds_test = ds_test.map(tokenize).remove_columns(['prompt'])


C:\Users\ctiha\Desktop\atmacup\atmaCup_env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\ctiha\Desktop\atmacup\atmaCup_env\Lib\site-packages\transformers\convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialize

In [9]:
# カスタム分類モデルの定義
class DebertaWithAdditionalFeatures(nn.Module):
    def __init__(self, deberta_model, num_additional_features):
        super(DebertaWithAdditionalFeatures, self).__init__()
        self.deberta = deberta_model
        self.fc1 = nn.Linear(self.deberta.config.hidden_size + num_additional_features, 256)
        self.fc2 = nn.Linear(256, 128)  # 二値分類のための出力層
        self.fc3 = nn.Linear(128, 1)  # 二値分類のための出力層
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()  # 二値分類のためのシグモイド活性化関数

    def forward(self, input_ids, attention_mask, additional_features):
        # DeBERTaモデルの出力を取得
        deberta_output = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = deberta_output.last_hidden_state[:, 0, :]  # CLSトークンの埋め込み

        # DeBERTaの出力と追加特徴量を結合
        combined_input = torch.cat((pooled_output, additional_features), dim=1)

        # 結合された特徴を全結合層に通す
        x = self.relu(self.fc1(combined_input))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# カスタムデータセットクラスの定義
class CustomDataset(Dataset):
    def __init__(self, texts, additional_features, labels):
        self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
        self.additional_features = additional_features
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['additional_features'] = torch.tensor(self.additional_features[idx], dtype=torch.float)
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)


# GPUが利用可能かを確認し、デバイスを設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

C:\Users\ctiha\Desktop\atmacup\atmaCup_env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Using device: cuda


In [10]:
train_texts, val_texts, train_features, val_features, train_labels, val_labels = train_test_split(
    train_texts, train_features, train_labels, test_size=0.3, stratify=train_labels
)

train_dataset = CustomDataset(train_texts, train_features, train_labels)
val_dataset = CustomDataset(val_texts, val_features, val_labels)

train_loader = DataLoader(train_dataset, batch_size=96, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


# カスタムモデルのインスタンス化
num_additional_features = len(train_features[0])
model = DebertaWithAdditionalFeatures(deberta_model, num_additional_features)

# モデルをGPUに移動
model.to(device)

# トレーニング用のハイパーパラメータ設定
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = nn.BCELoss()  # バイナリクロスエントロピー損失

# トレーニングループの実行
model.train()
for epoch in range(100):
    for batch in train_loader:
        optimizer.zero_grad()

        # バッチデータをGPUに移動
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        additional_features = batch['additional_features'].to(device)
        labels = batch['labels'].unsqueeze(1).to(device)  # ラベルもGPUに移動

        # モデルの出力と損失計算
        outputs = model(input_ids, attention_mask, additional_features)
        loss = criterion(outputs, labels)

        # 逆伝播とパラメータの更新
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


    # 評価ループの追加
    model.eval()  # 評価モードに切り替え
    all_labels = []
    all_preds = []
    all_probs = []

    with torch.no_grad():  # 勾配を計算しない
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            additional_features = batch['additional_features'].to(device)
            labels = batch['labels'].unsqueeze(1).to(device)

            # モデルの予測
            outputs = model(input_ids, attention_mask, additional_features)
            probs = torch.sigmoid(outputs).cpu().numpy()  # 確率に変換し、CPUに移動
            preds = (probs > 0.5).astype(int)  # 0.5を閾値としてクラスに変換

            # 結果を保存
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs)
            all_preds.extend(preds)

    # 評価指標の計算
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    auc = roc_auc_score(all_labels, all_probs)

    print(f"Validation - Epoch {epoch + 1}: Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

    model.train()  # モデルをトレーニングモードに戻す
    print("----------")


C:\Users\ctiha\AppData\Local\Temp\ipykernel_17984\673803681.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1, Loss: 0.33298417925834656
Validation - Epoch 1: Accuracy: 0.8203, Precision: 0.8203, Recall: 1.0000, F1: 0.9013, AUC: 0.9470
----------


C:\Users\ctiha\AppData\Local\Temp\ipykernel_17984\673803681.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 2, Loss: 0.1613921821117401
Validation - Epoch 2: Accuracy: 0.8203, Precision: 0.8203, Recall: 1.0000, F1: 0.9013, AUC: 0.9617
----------


C:\Users\ctiha\AppData\Local\Temp\ipykernel_17984\673803681.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


...

Epoch 98, Loss: 0.008765188045799732
Validation - Epoch 98: Accuracy: 0.8203, Precision: 0.8203, Recall: 1.0000, F1: 0.9013, AUC: 0.9591
----------


C:\Users\ctiha\AppData\Local\Temp\ipykernel_17984\673803681.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 99, Loss: 0.06528124958276749
Validation - Epoch 99: Accuracy: 0.8203, Precision: 0.8203, Recall: 1.0000, F1: 0.9013, AUC: 0.9552
----------


C:\Users\ctiha\AppData\Local\Temp\ipykernel_17984\673803681.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 100, Loss: 0.006524467375129461
Validation - Epoch 100: Accuracy: 0.8203, Precision: 0.8203, Recall: 1.0000, F1: 0.9013, AUC: 0.9584
----------


In [ ]:
# 推論
predictions = trainer.predict(test_dataset)
# ロジットを取得し、シグモイド関数で確率に変換
logits = predictions.predictions
probs = torch.sigmoid(torch.tensor(logits))
sample_submission["target"] = probs[:,1]

In [ ]:
sample_submission.to_csv("output/baseline.csv",index=False)